In [3]:
# Colab: mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Step 1: Initial cleansing**

In [4]:
# load pandas package and data
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/food_wishes_from2012.csv')
# check what the data looks like
df.head(3)

,Unnamed: 0,video_id,video_title,upload_date,view_count,like_count,comment_count
0,0,CQPLo8hECWg,Twice Baked Potatoes -- How to Make Fancy Stuf...,2012-12-15,4194592,47213,1950
1,1,TsrTU3CJn2c,Irish Shepherd's Pie - Classic Shepherd Pie fo...,2012-03-05,2730866,44230,2843
2,2,wRtGM3f-UBc,How to Flip Food in a Pan Like a Chef!,2012-10-04,2764323,17837,1382


In [5]:
# looks like there's an unwanted column at the very start
# need to remove it
df.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
# check the data types of columns
df.dtypes

video_id         object
video_title      object
upload_date      object
view_count        int64
like_count        int64
comment_count     int64
dtype: object

In [7]:
# the 'upload date' coulum should be in 'datetime' format but is currently showing as 'object'
# need to parse it to the correct format
df['upload_date'] = pd.to_datetime(df['upload_date'], format='%Y-%m-%d')

In [8]:
# also would be nice to single out the year, month , year-month and day only for finer analyses
df['upload_year'] = df['upload_date'].dt.year
df['upload_month'] = df['upload_date'].dt.month
df['upload_day'] = df['upload_date'].dt.day
df['upload_yearmonth'] = df['upload_date'].dt.to_period('M')

In [9]:
# check if there are any duplicate records in this dataframe
if df.shape[0] - len(df['video_id'].unique()) > 0:
    print('you have duplicates')
else:
    print('you don\'t have duplicates')

you have duplicates


In [10]:
# looks like there are duplicates
# now need to see what the duplicate records look like
df['duplicates'] = df['video_id'].duplicated(keep=False)
duplicates = df[df['duplicates'] == True]
print(f'there are {duplicates.shape[0]} records in the duplicates dataframe')
duplicates.groupby(duplicates.columns.values[0]).count()

there are 28 records in the duplicates dataframe


,video_title,upload_date,view_count,like_count,comment_count,upload_year,upload_month,upload_day,upload_yearmonth,duplicates
video_id,,,,,,,,,,
4i-blr330n4,2,2,2,2,2,2,2,2,2,2
8SLbT652IiQ,2,2,2,2,2,2,2,2,2,2
DE8EnTymPso,2,2,2,2,2,2,2,2,2,2
EVuiTbde80k,2,2,2,2,2,2,2,2,2,2
KHxlKnIS7pY,2,2,2,2,2,2,2,2,2,2
RCqFYhip2Ys,2,2,2,2,2,2,2,2,2,2
WYjsqeLHS14,2,2,2,2,2,2,2,2,2,2
dZ_3xKZMOec,2,2,2,2,2,2,2,2,2,2
dg-Wd3dNp6Q,2,2,2,2,2,2,2,2,2,2


In [11]:
# well, looks like some how the API queries gathered 2 exactly identical records for each of the 14 videos
# then just keep only one of each unique records
df['duplicates'] = df['video_id'].duplicated(keep='last')
df = df[df['duplicates'] == False]
df.drop(columns=['duplicates'], inplace=True)
# confirm if duplicates are removed
df.shape[0]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


1118

In [12]:
# finally, see what the cleaned-up dataframe looks like
df.head(3)

,video_id,video_title,upload_date,view_count,like_count,comment_count,upload_year,upload_month,upload_day,upload_yearmonth
0,CQPLo8hECWg,Twice Baked Potatoes -- How to Make Fancy Stuf...,2012-12-15,4194592,47213,1950,2012,12,15,2012-12
1,TsrTU3CJn2c,Irish Shepherd's Pie - Classic Shepherd Pie fo...,2012-03-05,2730866,44230,2843,2012,3,5,2012-03
2,wRtGM3f-UBc,How to Flip Food in a Pan Like a Chef!,2012-10-04,2764323,17837,1382,2012,10,4,2012-10


In [13]:
import numpy as np
df['month_diff'] = round((np.datetime64('2022-03-01') - pd.to_datetime(df['upload_yearmonth'].astype(str)))/np.timedelta64(1, 'M'), 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
counts_sum = df.groupby(['upload_yearmonth', 'month_diff']).sum()[['view_count', 'like_count', 'comment_count']].reset_index()
counts_sum['upload_yearmonth'] = counts_sum['upload_yearmonth'].astype(str)
for col in list(counts_sum.columns.values)[2:5]:
        new_col = col + '_month_normalised'
        counts_sum[new_col] = counts_sum[col] / counts_sum['month_diff']

counts_sum.head(3)

,upload_yearmonth,month_diff,view_count,like_count,comment_count,view_count_month_normalised,like_count_month_normalised,comment_count_month_normalised
0,2012-01,122.0,6046941,96394,8131,49565.090164,790.114754,66.647541
1,2012-02,121.0,16977317,210647,11742,140308.404959,1740.884298,97.041322
2,2012-03,120.0,9596025,169435,10616,79966.875000,1411.958333,88.466667


In [15]:
import plotly.express as px

for col in counts_sum.columns.values[2:5]:
    fig = px.bar(counts_sum, x='upload_yearmonth', y=col)
    fig.show()

In [16]:
for col in counts_sum.columns.values[5:8]:
    fig = px.bar(counts_sum, x='upload_yearmonth', y=col)
    fig.show()